## Visualizing taxi trajectories with MovingPandas

In [84]:
import datetime
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point

In [85]:
input_file_path = "../data/raw_data/subset-10000-6-percent.csv"

### Create dataframe

In [86]:
# Map gets a bit slow above 100 rows
# Choose how many rows to include
# Skip the desired amount to look at different IDs

df = pd.read_csv(input_file_path, nrows=60, skiprows=[i for i in range(1, 100)])
df.POLYLINE = df.POLYLINE.apply(eval)  # string to list

### Some helper functions
We need to create a DataFrame with one point + timestamp per row before we can use MovingPandas to create Trajectories

In [87]:
def unixtime_to_datetime(unix_time):
    return datetime.datetime.fromtimestamp(unix_time)
 
def compute_datetime(row):
    unix_time = row['TIMESTAMP']
    offset = row['running_number'] * datetime.timedelta(seconds=15)
    return unixtime_to_datetime(unix_time) + offset
 
def create_point(xy):
    try: 
        return Point(xy)
    except TypeError:  # when there are nan values in the input data
        return None

In [88]:
new_df = df.explode('POLYLINE')
new_df['geometry'] = new_df['POLYLINE'].apply(create_point)
new_df['running_number'] = new_df.groupby('TRIP_ID').cumcount()
new_df['datetime'] = new_df.apply(compute_datetime, axis=1)
new_df.drop(columns=['POLYLINE', 'TIMESTAMP', 'running_number'], inplace=True)

In [89]:
trajs = mpd.TrajectoryCollection(gpd.GeoDataFrame(new_df, crs=4326), traj_id_col='TRIP_ID', obj_id_col='TAXI_ID', t='datetime')
trajs.hvplot(title='Taxi Trajectory Data', tiles='CartoLight')

:Overlay
   .WMTS.I         :WMTS   [Longitude,Latitude]
   .Path.I         :Path   [Longitude,Latitude]
   .Path.II        :Path   [Longitude,Latitude]
   .Path.III       :Path   [Longitude,Latitude]
   .Path.IV        :Path   [Longitude,Latitude]
   .Path.V         :Path   [Longitude,Latitude]
   .Path.VI        :Path   [Longitude,Latitude]
   .Path.VII       :Path   [Longitude,Latitude]
   .Path.VIII      :Path   [Longitude,Latitude]
   .Path.IX        :Path   [Longitude,Latitude]
   .Path.X         :Path   [Longitude,Latitude]
   .Path.XI        :Path   [Longitude,Latitude]
   .Path.XII       :Path   [Longitude,Latitude]
   .Path.XIII      :Path   [Longitude,Latitude]
   .Path.XIV       :Path   [Longitude,Latitude]
   .Path.XV        :Path   [Longitude,Latitude]
   .Path.XVI       :Path   [Longitude,Latitude]
   .Path.XVII      :Path   [Longitude,Latitude]
   .Path.XVIII     :Path   [Longitude,Latitude]
   .Path.XIX       :Path   [Longitude,Latitude]
   .Path.XX        :Path   [Longitude,Latitude]
   .Path.XXI       :Path   [Longitude,Latitude]
   .Path.XXII      :Path   [Longitude,Latitude]
   .Path.XXIII     :Path   [Longitude,Latitude]
   .Path.XXIV      :Path   [Longitude,Latitude]
   .Path.XXV       :Path   [Longitude,Latitude]
   .Path.XXVI      :Path   [Longitude,Latitude]
   .Path.XXVII     :Path   [Longitude,Latitude]
   .Path.XXVIII    :Path   [Longitude,Latitude]
   .Path.XXIX      :Path   [Longitude,Latitude]
   .Path.XXX       :Path   [Longitude,Latitude]
   .Path.XXXI      :Path   [Longitude,Latitude]
   .Path.XXXII     :Path   [Longitude,Latitude]
   .Path.XXXIII    :Path   [Longitude,Latitude]
   .Path.XXXIV     :Path   [Longitude,Latitude]
   .Path.XXXV      :Path   [Longitude,Latitude]
   .Path.XXXVI     :Path   [Longitude,Latitude]
   .Path.XXXVII    :Path   [Longitude,Latitude]
   .Path.XXXVIII   :Path   [Longitude,Latitude]
   .Path.XXXIX     :Path   [Longitude,Latitude]
   .Path.XL        :Path   [Longitude,Latitude]
   .Path.XLI       :Path   [Longitude,Latitude]
   .Path.XLII      :Path   [Longitude,Latitude]
   .Path.XLIII     :Path   [Longitude,Latitude]
   .Path.XLIV      :Path   [Longitude,Latitude]
   .Path.XLV       :Path   [Longitude,Latitude]
   .Path.XLVI      :Path   [Longitude,Latitude]
   .Path.XLVII     :Path   [Longitude,Latitude]
   .Path.XLVIII    :Path   [Longitude,Latitude]
   .Path.XLIX      :Path   [Longitude,Latitude]
   .Path.L         :Path   [Longitude,Latitude]
   .Path.LI        :Path   [Longitude,Latitude]
   .Path.LII       :Path   [Longitude,Latitude]
   .Path.LIII      :Path   [Longitude,Latitude]
   .Path.LIV       :Path   [Longitude,Latitude]
   .Path.LV        :Path   [Longitude,Latitude]
   .Path.LVI       :Path   [Longitude,Latitude]
   .Path.LVII      :Path   [Longitude,Latitude]
   .Path.LVIII     :Path   [Longitude,Latitude]
   .Path.LIX       :Path   [Longitude,Latitude]
   .Path.LX        :Path   [Longitude,Latitude]
   .Points.I       :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.III     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.IV      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.V       :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VI      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VII     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VIII    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.IX      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.X       :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XI      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XII     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XIII    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XIV     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XV      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XVI     :Points  